In [1]:
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")
import os

physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for gpu in physical_devices:
        tf.config.experimental.set_memory_growth(gpu,True)

2024-11-03 06:30:52.752422: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-03 06:30:52.916214: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-03 06:30:52.963607: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-03 06:30:53.196675: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
I0000 00:00:1730615462.994155      12 cuda_executor.c

In [8]:
import numpy as np
import tensorflow as tf
import polars as pl

# Load the data (assuming your data files are in the correct directory as specified)
directory = '/app/data/filteredData'
balltoball = pl.read_csv(os.path.join(directory, 'balltoball.csv'))
teamStats = pl.read_csv(os.path.join(directory, 'team12Stats.csv'))
playsrStats = np.load(os.path.join(directory, 'playerStats.npy'), allow_pickle=True)
balltoball

match_id,innings,ball,curr_score,curr_wickets,won
i64,i64,f64,i64,i64,i64
211028,1,0.1,0,0,1
211028,1,0.2,1,0,1
211028,1,0.3,1,0,1
211028,1,0.4,1,0,1
211028,1,0.5,1,0,1
…,…,…,…,…,…
1450765,2,19.3,100,9,0
1450765,2,19.4,100,9,0
1450765,2,19.5,104,9,0


In [7]:
teamStats

match_id,gender,Mat,Won,Lost,Tied,NR,W/L,Inns,HS,LS,AveRPW,AveRPO,Mat_team2,Won_team2,Lost_team2,Tied_team2,NR_team2,W/L_team2,Inns_team2,HS_team2,LS_team2,AveRPW_team2,AveRPO_team2
i64,i64,i64,i64,i64,i64,i64,f64,i64,i64,f64,f64,f64,i64,i64,i64,i64,i64,f64,i64,i64,f64,f64,f64
211028,1,1,1,0,0,0,1.0,1,179,0.0,22.37,8.95,1,0,1,0,0,0.0,1,79,79.0,7.9,5.44
211028,1,1,0,1,0,0,0.0,1,79,79.0,7.9,5.44,1,1,0,0,0,1.0,1,179,0.0,22.37,8.95
211048,1,1,0,1,0,0,0.0,1,170,170.0,17.0,8.5,1,1,0,0,0,1.0,1,214,0.0,42.8,10.7
211048,1,1,1,0,0,0,1.0,1,214,0.0,42.8,10.7,1,0,1,0,0,0.0,1,170,170.0,17.0,8.5
225263,1,2,0,2,0,0,0.0,2,161,0.0,25.41,7.62,1,1,0,0,0,1.0,1,148,0.0,29.6,8.29
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1450753,1,5,5,0,0,0,5.0,5,183,119.0,21.28,7.81,5,0,5,0,0,0.0,5,57,18.0,3.56,2.67
1450759,1,5,0,5,0,0,0.0,5,57,18.0,3.56,2.67,10,6,4,0,0,1.5,10,176,100.0,26.9,7.11
1450759,1,10,6,4,0,0,1.5,10,176,100.0,26.9,7.11,5,0,5,0,0,0.0,5,57,18.0,3.56,2.67


In [11]:
playsrStats

array([[[2.11028000e+05, 1.00000000e+00, 1.00000000e+00, ...,
         2.00000000e+00, 0.00000000e+00, 2.00000000e+00],
        [2.11028000e+05, 1.00000000e+00, 1.00000000e+00, ...,
         1.00000000e+00, 0.00000000e+00, 1.00000000e+00],
        [2.11028000e+05, 1.00000000e+00, 1.00000000e+00, ...,
         1.00000000e+00, 0.00000000e+00, 1.00000000e+00],
        ...,
        [2.11048000e+05, 1.00000000e+00, 1.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [2.11048000e+05, 1.00000000e+00, 1.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [2.11048000e+05, 1.00000000e+00, 1.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

       [[2.25263000e+05, 1.00000000e+00, 1.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [2.25263000e+05, 1.00000000e+00, 1.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [2.25263000e+05, 

In [10]:
balltoball.shape , teamStats.shape , playsrStats.shape

((534910, 6), (2312, 24), (2312, 22, 23))